Restructuring the files assuming that a folder called "train" and a file called "train.csv" exist in a directory called "CactusImages"

In [23]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
from shutil import copyfile
from keras.models import Sequential
ORIGIN_PATH=os.getcwd()
TRAIN_PATH = os.path.abspath("aerial-cactus-identification/train/")
TEST_PATH = os.path.abspath("aerial-cactus-identification/test/")
META_PATH = os.path.abspath("aerial-cactus-identification/train.csv")
GENERATOR_PATH = os.path.abspath("data/")

In [4]:
metaData = pd.read_csv(META_PATH)
try :
    os.mkdir("data")
    os.chdir("./data/")
    os.mkdir("has_cactus_1")
    os.mkdir("has_cactus_0")
    os.chdir(ORIGIN_PATH)
except :
    pass

In [ ]:
destination_1_path = os.path.join(ORIGIN_PATH,"data","has_cactus_1")
destination_0_path = os.path.join(ORIGIN_PATH,"data","has_cactus_0")
total = len(metaData.values)
progress=0.0
for row in metaData.values:
    progress+=1
    file = row[0]
    label = row[1]
    file_path = os.path.join(TRAIN_PATH,file)
    if label == 1:
        dest_path = os.path.join(destination_1_path,file)
        os.rename(file_path,dest_path)
    else :
        dest_path = os.path.join(destination_0_path,file)
        copyfile(file_path,dest_path)

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

In [12]:
li= os.listdir(destination_0_path)

In [24]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   validation_split=0.3,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_set = train_datagen.flow_from_directory(GENERATOR_PATH,
                                                 target_size = (244, 244),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='training')
valiation_set = train_datagen.flow_from_directory(GENERATOR_PATH,
                                                 target_size = (244, 244),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='validation')

Found 12251 images belonging to 2 classes.
Found 5249 images belonging to 2 classes.


In [15]:
classifier.fit_generator(train_set,
                           steps_per_epoch=345,
                           epochs=10,
                           validation_data=test_set,
                           validation_steps=86)

1
3
5


StopIteration: 